# Aula 5 Funçoes e estruturas

## Imports

In [80]:
import pandas as pd
import numpy as np
from geopy.geocoders import Nominatim
import plotly.express as px

## Loads

In [81]:
df=pd.read_csv('datasets/kc_house_data.csv')

## Data dimensions

In [82]:
print('Number of rows:{}'.format(df.shape[0]))
print('Number of cols:{}'.format(df.shape[1]))

Number of rows:21613
Number of cols:21


## Convert data

In [83]:
df['date']=pd.to_datetime(df['date'])
df.dtypes

id                        int64
date             datetime64[ns]
price                   float64
bedrooms                  int64
bathrooms               float64
sqft_living               int64
sqft_lot                  int64
floors                  float64
waterfront                int64
view                      int64
condition                 int64
grade                     int64
sqft_above                int64
sqft_basement             int64
yr_built                  int64
yr_renovated              int64
zipcode                   int64
lat                     float64
long                    float64
sqft_living15             int64
sqft_lot15                int64
dtype: object

## Statistics descriptions

In [84]:
num_attributes=df.select_dtypes(include=('int64','float64'))

In [85]:
print('Number of rows:{}'.format(num_attributes.shape[1]))
num_attributes.dtypes

Number of rows:20


id                 int64
price            float64
bedrooms           int64
bathrooms        float64
sqft_living        int64
sqft_lot           int64
floors           float64
waterfront         int64
view               int64
condition          int64
grade              int64
sqft_above         int64
sqft_basement      int64
yr_built           int64
yr_renovated       int64
zipcode            int64
lat              float64
long             float64
sqft_living15      int64
sqft_lot15         int64
dtype: object

In [86]:
pd.set_option('display.float_format', lambda x: f'%.{len(str(x%1))-2}f' % x) #tira a notaçao cientifica

media=pd.DataFrame(num_attributes.apply(np.mean, axis=0))
mediana=pd.DataFrame(num_attributes.apply(np.median,axis=0))

maximo=pd.DataFrame(num_attributes.apply(np.max,axis=0))
minimo=pd.DataFrame(num_attributes.apply(np.min,axis=0))
std=pd.DataFrame(num_attributes.apply(np.std,axis=0))

df1=pd.concat([maximo,minimo,media,mediana,std], axis=1).reset_index()
df1.columns=['attributes','maximo','minimo','media','mediana','std']

In [87]:
df1

,attributes,maximo,minimo,media,mediana,std
0,id,9900000190.0,1000102.0,4580301520.8649883270263672,3904930410.0,2876499023.4277825355529785
1,price,7700000.0,75000.0,540088.14176652941387147,450000.0,367118.7031813723151572
2,bedrooms,33.0,0.0,3.3708416230972098,3.0,0.9300403146391111
3,bathrooms,8.0,0.0,2.1147573219821405,2.25,0.7701453398840542
4,sqft_living,13540.0,290.0,2079.8997362698373763,1910.0,918.41964938288231224
5,sqft_lot,1651359.0,520.0,15106.9675658168689552,7618.0,41419.5532726276796893
6,floors,3.5,1.0,1.49430898070605656,1.5,0.5399764027733692
7,waterfront,1.0,0.0,0.007541757275713691,0.0,0.08651519619643655
8,view,4.0,0.0,0.23430342849211122,0.0,0.7662998409056494
9,condition,5.0,1.0,3.40942951001711947,3.0,0.6507279917561631


### Crie uma nova coluna chamada 'dormitory_type'
- se o valor dacoluna 'bedrooms' for igual a 1 = studio
- se o valor dacoluna 'bedrooms' for igual a 2 = apartment
- se o valor dacoluna 'bedrooms' for igual a 3 = house

In [88]:
df['dormitory_type']='NA'

for i in range(len(df)):
    if df.loc[i,'bedrooms']==1:
        df.loc[i,'dormitory_type']='studio'
    elif  df.loc[i,'bedrooms']==2:
        df.loc[i,'dormitory_type']='apartment'
    else:
        df.loc[i,'dormitory_type']='house'
    

### Qual a quantidade de imóveis por nível?
- Nivel 0: Preço entre R$ 0.00 e R$ 321.950
- Nivel 1: Preço entre R$ 321.950 e R$ 450.000
- Nivel 2: Preço entre R$ 450.000 e R$ 645.000
- Nivel 3: Preço entre R$ Acima de R$ 645.000

In [89]:
df['level']='NA'

for i in range(len(df)):
    if (df.loc[i,'price']>=0)&(df.loc[i,'price']<321950):
        df.loc[i,'level']=1
    elif (df.loc[i,'price']>=321950)&(df.loc[i,'price']<450000):
        df.loc[i,'level']=2
    elif (df.loc[i,'price']>=450000)&(df.loc[i,'price']<645000):
        df.loc[i,'level']=3
    else:
        df.loc[i,'level']=4

In [90]:
#initialize API
geolocator=Nominatim(user_agent='geoapiExercises')
df=df.head(1000)#just sample

#create empty columns
df.loc[:,'road']='NA'
df.loc[:,'house_number']='NA'
for i in range(len(df)):
    print('Loop:{}/{}'.format(i,len(df)))
    
    query=str(df.loc[i,'lat'])+','+str(df.loc[i,'long'])
    response=geolocator.reverse(query)
    
    if 'house_number' in response.raw['address']:
        df.loc[i,'house_number']=response.raw['address']['house_number']
    if 'road' in response.raw['address']:
        df.loc[i,'road']        =response.raw['address']['road']


Loop:0/1000
Loop:1/1000
Loop:2/1000
Loop:3/1000
Loop:4/1000
Loop:5/1000
Loop:6/1000
Loop:7/1000
Loop:8/1000
Loop:9/1000
Loop:10/1000
Loop:11/1000
Loop:12/1000
Loop:13/1000
Loop:14/1000
Loop:15/1000
Loop:16/1000
Loop:17/1000
Loop:18/1000
Loop:19/1000
Loop:20/1000
Loop:21/1000
Loop:22/1000
Loop:23/1000
Loop:24/1000
Loop:25/1000
Loop:26/1000
Loop:27/1000
Loop:28/1000
Loop:29/1000
Loop:30/1000
Loop:31/1000
Loop:32/1000
Loop:33/1000
Loop:34/1000
Loop:35/1000
Loop:36/1000
Loop:37/1000
Loop:38/1000
Loop:39/1000
Loop:40/1000
Loop:41/1000
Loop:42/1000
Loop:43/1000
Loop:44/1000
Loop:45/1000
Loop:46/1000
Loop:47/1000
Loop:48/1000
Loop:49/1000
Loop:50/1000
Loop:51/1000
Loop:52/1000
Loop:53/1000
Loop:54/1000
Loop:55/1000
Loop:56/1000
Loop:57/1000
Loop:58/1000
Loop:59/1000
Loop:60/1000
Loop:61/1000
Loop:62/1000
Loop:63/1000
Loop:64/1000
Loop:65/1000
Loop:66/1000
Loop:67/1000
Loop:68/1000
Loop:69/1000
Loop:70/1000
Loop:71/1000
Loop:72/1000
Loop:73/1000
Loop:74/1000
Loop:75/1000
Loop:76/1000
Loop:77/1

Loop:594/1000
Loop:595/1000
Loop:596/1000
Loop:597/1000
Loop:598/1000
Loop:599/1000
Loop:600/1000
Loop:601/1000
Loop:602/1000
Loop:603/1000
Loop:604/1000
Loop:605/1000
Loop:606/1000
Loop:607/1000
Loop:608/1000
Loop:609/1000
Loop:610/1000
Loop:611/1000
Loop:612/1000
Loop:613/1000
Loop:614/1000
Loop:615/1000
Loop:616/1000
Loop:617/1000
Loop:618/1000
Loop:619/1000
Loop:620/1000
Loop:621/1000
Loop:622/1000
Loop:623/1000
Loop:624/1000
Loop:625/1000
Loop:626/1000
Loop:627/1000
Loop:628/1000
Loop:629/1000
Loop:630/1000
Loop:631/1000
Loop:632/1000
Loop:633/1000
Loop:634/1000
Loop:635/1000
Loop:636/1000
Loop:637/1000
Loop:638/1000
Loop:639/1000
Loop:640/1000
Loop:641/1000
Loop:642/1000
Loop:643/1000
Loop:644/1000
Loop:645/1000
Loop:646/1000
Loop:647/1000
Loop:648/1000
Loop:649/1000
Loop:650/1000
Loop:651/1000
Loop:652/1000
Loop:653/1000
Loop:654/1000
Loop:655/1000
Loop:656/1000
Loop:657/1000
Loop:658/1000
Loop:659/1000
Loop:660/1000
Loop:661/1000
Loop:662/1000
Loop:663/1000
Loop:664/1000
Loop:6

In [91]:
response=geolocator.reverse('47.5112,-122.257')
response

Location(10012, 61st Avenue South, Rainier Beach, Seattle, King County, Washington, 98178, United States, (47.5112302, -122.25676111324441, 0.0))

In [92]:
response.raw

{'place_id': 147183522,
 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright',
 'osm_type': 'way',
 'osm_id': 236673600,
 'lat': '47.5112302',
 'lon': '-122.25676111324441',
 'display_name': '10012, 61st Avenue South, Rainier Beach, Seattle, King County, Washington, 98178, United States',
 'address': {'house_number': '10012',
  'road': '61st Avenue South',
  'neighbourhood': 'Rainier Beach',
  'city': 'Seattle',
  'county': 'King County',
  'state': 'Washington',
  'postcode': '98178',
  'country': 'United States',
  'country_code': 'us'},
 'boundingbox': ['47.511189', '47.5112943', '-122.2568571', '-122.2566651']}

In [93]:
df

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15,dormitory_type,level,road,house_number
0,7129300520,2014-10-13,221900.0,3,1.0,1180,5650,1.0,0,0,...,0,98178,47.5112000000000023,-122.257000000000005,1340,5650,house,1,61st Avenue South,10012
1,6414100192,2014-12-09,538000.0,3,2.25,2570,7242,2.0,0,0,...,1991,98125,47.7209999999999965,-122.3190000000000026,1690,7639,house,3,Northeast 127th Street,837
2,5631500400,2015-02-25,180000.0,2,1.0,770,10000,1.0,0,0,...,0,98028,47.7379000000000033,-122.2330000000000041,2720,8062,apartment,1,81st Avenue Northeast,15109
3,2487200875,2014-12-09,604000.0,4,3.0,1960,5000,1.0,0,0,...,0,98136,47.5208000000000013,-122.3930000000000007,1360,5000,house,3,Fauntleroy Way Southwest,9236
4,1954400510,2015-02-18,510000.0,3,2.0,1680,8080,1.0,0,0,...,0,98074,47.6167999999999978,-122.0450000000000017,1800,7503,house,3,221st Avenue Northeast,923
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,7437100570,2014-08-21,291000.0,4,2.5,1860,6325,2.0,0,0,...,0,98038,47.3492000000000033,-122.0300000000000011,1860,6449,house,1,234th Avenue Southeast,28127
996,8856004730,2014-09-17,199950.0,2,2.75,1590,20917,1.5,0,0,...,0,98001,47.2785999999999973,-122.25,1310,6000,apartment,1,Milwaukee Boulevard South,119
997,3856902996,2014-08-04,553500.0,2,1.0,850,2340,1.0,0,0,...,0,98105,47.6706999999999965,-122.328000000000003,1300,3000,apartment,3,Northeast 58th Street,101
998,1442800370,2015-04-15,189950.0,2,1.0,1030,4188,1.0,0,0,...,0,98038,47.3738000000000028,-122.0570000000000022,1450,3376,apartment,1,213th Avenue Southeast,25423


In [94]:
#map
houses=df[['id','long', 'lat', 'price','level']].copy()

fig=px.scatter_mapbox(houses,
                  lat='lat',
                  lon='long',
                  color='level',
                  size='price',
                  color_continuous_scale=px.colors.cyclical.IceFire,
                  size_max=15,
                  zoom=10)

fig.update_layout(mapbox_style='open-street-map')
fig.update_layout(height=300,margin={'r':0,'t':0,'l':0,'b':0})
fig.show()

# Refazendo o codigo